In [ ]:
import pandas as pd
import numpy as np
import hrvanalysis as hrv
import matplotlib.pyplot as plt
import seaborn as sb
import glob
import scipy.stats as ss
import peakutils as pu

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sb.set(font_scale=1)

In [ ]:
ecg_data = sorted(glob.glob('../data/Me/*ECG*', recursive=True))[-1]
df_ecg = pd.read_csv(ecg_data, header=None, sep=' ', names=['x', 'y'], index_col=0, dtype=np.int)
thres = 550 # (np.median(df_ecg) + np.diff(np.quantile(df_ecg, [0.25, 0.75]))[0] * 1.5)
df_peaks = df_ecg.iloc[pu.indexes(df_ecg.values.ravel(), thres=thres, min_dist=350, thres_abs=True)]

In [ ]:
fn = ecg_data.replace(' ECG.txt', ' peaks.txt')
df_peaks.to_csv(fn, header=False, sep=' ')

In [ ]:
plt.figure(figsize=(13.5,4))
#plt.ylim(400, 950)
plt.xlim(df_ecg.index.min(), df_ecg.index.max())
plt.plot(df_ecg.index.values, df_ecg.values.ravel(), lw=1)
plt.plot(df_peaks.index.values, df_peaks.values.ravel(), 'ro', ms=3)

## RR intervals

In [ ]:
rr = np.diff(df_peaks.index)
sb.distplot(rr, kde=False, rug=True)

In [ ]:
plt.ylim(0, 1300)
sb.lineplot(range(len(rr)), rr, lw=1)

In [ ]:
nn = np.array(hrv.get_nn_intervals(rr))
nn = nn[~np.isnan(nn)]

## Features 

In [ ]:
fdf = hrv.get_frequency_domain_features(nn)
fdf

In [ ]:
ccf = hrv.get_csi_cvi_features(nn)
ccf

In [ ]:
tdf = hrv.get_time_domain_features(nn)
tdf

In [ ]:
hrv.get_poincare_plot_features(nn)

# HRV score and Stress index (Baevsky, 2009)

In [ ]:
hrv_score = 20*np.log(tdf['rmssd'])

mode = ss.mode(rr)[0][0]
hist = np.histogram(nn, bins=np.arange(0, 2500, 50))
mid1 = np.argwhere(hist[1] > mode)[0][0] - 1
mid2 = mid1 + 1
amo = (hist[0][mid1:mid2].item() / len(nn)) * 100
isb = amo / (2 * mode) / (np.max(nn) - np.min(nn)) * 1000 * 1000

hrv_score, isb

In [ ]:
hrv.plot.plot_psd(rr)